### Import data after SQL-cleaning


----

In [1]:
import pandas as pd
import getpass
import sqlalchemy as sa




# Importing SQL- cleaned data into pandas

In [10]:

driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'

connection_string = f'{driver}://{user}:{password}@{ip}'
    
db_connection = sa.create_engine(connection_string)

insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()


Your password:········


## Check Olist tables concerning missing or misleading data

In [11]:
query = "SELECT * FROM olist.closed_deals"
closed_deals = pd.read_sql_query(query, db_connection)
closed_deals.isnull().sum()


##cleaning
#delete (no value)
    #sdr_id 
    #sr_id 
#delete (too many nulls)
    #lead_behaviour_profile           
    #has_company                      
    #has_gtin                         
    #average_stock
    #declared_product_catalog_size 

##interesting
# time to close deal
# closed_deals per time (quarter/year)
# percentage buisness_segment and business_type

mql_id                             0
seller_id                          0
sdr_id                             0
sr_id                              0
won_date                           0
business_segment                   1
lead_type                          6
lead_behaviour_profile           177
has_company                      779
has_gtin                         778
average_stock                    776
business_type                     10
declared_product_catalog_size    773
declared_monthly_revenue           0
dtype: int64

In [13]:
query = "SELECT * FROM olist.marketing_qualified_leads"
marketing_qualified_leads = pd.read_sql_query(query, db_connection)
marketing_qualified_leads.isnull().sum()

##cleaning
# null cells into "unknown"

## interesting (related to closed_deals)
# top 10 landing_page
# origin (also by time to close)

mql_id                 0
first_contact_date     0
landing_page_id        0
origin                60
dtype: int64

In [22]:
query = "SELECT * FROM olist.`sellers`"
sellers = pd.read_sql_query(query, db_connection)
sellers.isnull().sum()

##cleaning
#seller_city one has a number = rio de janeiro
#seller state = full name

## interesting
#seller city
#seller state 

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

In [15]:
query = "SELECT * FROM olist.order_items"
order_items = pd.read_sql_query(query, db_connection)
order_items.isnull().sum()

## findings
# freight_value & price & final_price are the prices for the customer (and is the same as in other tables)
# order_item_id has  21 uniques -->  shows how many items are included in one order shipped out to the customer
# shipping_limit_date = latest time when the seller has to give the product to the carrier for delivery

##interesting
#which seller violate the shipping_limit_date (shipping_date_limit<order_delivered_carrier_date)

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [18]:
query = "SELECT * FROM olist.orders"
orders = pd.read_sql_query(query, db_connection)
orders.isnull().sum()

##cleaning
#bucketing "processing & approved, invoiced"  as these orders have not been processed correctly: new name "unprocessed"; rename "shipped" "lost_during_delivery"

##interesting
# percentage order status 
# actual vs esimated delivery date (only orders who have a order_delivered_customer_date)

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [16]:
query = "SELECT * FROM olist.order_payments"
order_payments = pd.read_sql_query(query, db_connection)
order_payments.isnull().sum()

## cleanining
#payment_type = delete rows where type = 'not_defined' 
#payment_sequential = no clear added value

## Interesting
#percentage of payment_type per month /year (reference order_delivered_customer_date)
#payment_installments: average payment_installments, zip_code analysis, category analysis

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [7]:
query = "SELECT * FROM olist.customers"
customers = pd.read_sql_query(query, db_connection)
customers.isnull().sum()

##cleaning
#customer state = full name

## interesting
#customer city
#customer state 

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [14]:
query = "SELECT * FROM olist.`order_customer_items_paid`"
order_customer_items_paid = pd.read_sql_query(query, db_connection)
order_customer_items_paid.isnull().sum()

##
#how many delivered orders haven't been paid yet (compare with order_ids in orders where status is 'delivered'):
 ''' Just one:
 SELECT *
from orders
Left join `order_customer_items_paid`
using (order_id)
where order_customer_items_paid.order_id is null and orders.order_status in ('shipped', 'delivered')

Output (order_id:) a68ce1686d536ca72bd2dadc4b8671e5

this order has a review , Übersetzung: "DIE LIEFERUNG SIND 38 TAGE ES SIND BEREITS MEHR ALS 60 TAGE UND ICH HABE DAS PRODUKT NICHT ERHALTEN" '''

##Interesting
#show percentage odrder_status (if order_status = "shipping" / "lost_during_delivery" --> order paid but not received)

order_id         0
customer_city    0
customer_id      0
order_status     0
order_item_id    0
price            0
freight_value    0
final_price      0
dtype: int64

In [17]:
query = "SELECT * FROM olist.order_reviews"
order_reviews = pd.read_sql_query(query, db_connection)
order_reviews.isnull().sum()

## interesting
# review_score
# top 10 seller score / low 10 seller score compare to their revenue per month/year 
# review answer velocity over time(month/year) (calculated review_answer_timestamp-review_creation_date)
# from which region come the most reviews / also by good and bad score

review_id                      0
order_id                       0
review_score                   0
review_comment_title       88285
review_comment_message     58245
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [19]:
query = "SELECT * FROM olist.`order_status_year_price`"
order_status_year_price = pd.read_sql_query(query, db_connection)
order_status_year_price.isnull().sum()
##findings
#only delivered 

##interesting
#to calculate revenue (compare it with order_customer_items_paid)

order_id                 0
order_item_id            0
order_status             0
Year                     0
product_id               0
product_category_name    0
category                 0
total_price              0
dtype: int64

In [21]:
query = "SELECT * FROM olist.`products`"
products = pd.read_sql_query(query, db_connection)
products.isnull().sum()

##cleaning
#change product_catgeory_name to english 

##interesting
# which seller haven't provided all of the necessary information 

product_id                      0
product_category_name         610
product_name_length           610
product_description_length    610
product_photos_qty            610
product_weight_g                6
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [20]:
query = "SELECT * FROM olist.`product_category_name_translation`"
product_category_name_translation = pd.read_sql_query(query, db_connection)
product_category_name_translation.isnull().sum()


product_category_name            0
product_category_name_english    0
dtype: int64

In [12]:
query = "SELECT * FROM olist.geolocation"
geolocation = pd.read_sql_query(query, db_connection)
geolocation.isnull().sum()

# we don't need this table as we are using tableau and it can create lattitude itself with the city information

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64